In [ ]:
# code below to merge the two separate data sets

In [59]:
# Time Series Data
# Load the datasets
covid_data = pd.read_csv("COVID-19 Time Series Data.csv")
gdp_data = pd.read_csv("gdp_per_capita.csv", delimiter=',', skiprows=3, on_bad_lines='warn', keep_default_na=True)
democracy_data = pd.read_csv("DemocracyMatrix_v4.csv")

# Convert the 'Date' column in the COVID dataset to datetime format
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

# Filter the COVID data for dates between January 22, 2020, and January 22, 2022
start_date = '2020-01-22'
end_date = '2022-01-22'
covid_filtered = covid_data[(covid_data['Date'] >= start_date) & (covid_data['Date'] <= end_date)]

# Extract the year from the 'Date' column in the COVID dataset
covid_filtered['Year'] = covid_filtered['Date'].dt.year

# Create 'Month' column from the 'Date' column
covid_filtered['Month'] = covid_filtered['Date'].dt.month

# Create 'Date' column from the 'Date' column
covid_filtered['Date'] = covid_filtered['Date'].dt.date

# Drop the 'Province/State' column since it's not needed
covid_filtered = covid_filtered.drop(columns=['Province/State'])

# Rename 'Country/Region' column
covid_filtered = covid_filtered.rename(columns={'Country/Region': 'Country'})

# Drop unnecessary columns
covid_filtered = covid_filtered.drop(columns = ['Confirmed', 'Recovered'])
# Export the final merged DataFrame to a CSV file
covid_filtered .to_csv('cleaned_covid_data.csv', index=False)




<ipython-input-59-c2935eb4a1ce>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_filtered['Year'] = covid_filtered['Date'].dt.year
<ipython-input-59-c2935eb4a1ce>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_filtered['Month'] = covid_filtered['Date'].dt.month
<ipython-input-59-c2935eb4a1ce>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [ ]:
# Dataset with COVID deaths, GDP, and democracy scores
# Filter COVID dataset to 2020
covid_2020 = covid_filtered[covid_filtered['Year'] == 2020]

# Reshape the GDP data to have a 'Year' column
gdp_melted = gdp_data.melt(id_vars='Country Name', value_vars=['2020', '2021', '2022'],
                           var_name='Year', value_name='GDP_per_Capita')

# Convert the 'Year' column to numeric
gdp_melted['Year'] = pd.to_numeric(gdp_melted['Year'])

# Rename 'Country Name' column
gdp_melted = gdp_melted.rename(columns={'Country Name': 'Country'})

# Merge the COVID dataset with the GDP dataset on 'Country' and 'Year'
merged_data = pd.merge(covid_2020, gdp_melted, how='inner', on=['Country', 'Year'])

# Rename columns in the democracy dataset
democracy_data = democracy_data.rename(columns={'country': 'Country', 'year': 'Year'})

# Filter democracy data for the year 2020 and select relevant columns
democracy_2020 = democracy_data[democracy_data['Year'] == 2020][['Country', 'Year',  'total_index_context', 'classification_context']]

# Ensure there are no leading spaces in 'Country' names
democracy_2020['Country'] = democracy_2020['Country'].str.strip()

# Merge the data with COVID and GDP with democracy scores on 'Country' and 'Year'
merged_data2 = pd.merge(merged_data, democracy_2020, how='inner', on=['Country', 'Year'])

# Group by 'Country' and 'Year'
grouped_data = merged_data2.groupby(['Country', 'Year']).agg({
    'Deaths': 'sum',  # sum deaths for each date in month
    'GDP_per_Capita': 'mean',  # Average GDP per capita
    'total_index_context': 'mean',  # Average democracy
    'classification_context': 'first', # Get the first classification context
}).reset_index()

# rename columns
grouped_data=grouped_data.rename(columns={'total_index_context': 'Democracy_Scores', 'classification_context': 'Democracy_classification'})

# Export the final merged DataFrame to a CSV file
grouped_data.to_csv('merged_covid_gdp_democracy.csv', index=False)





